In [9]:
import pandas as pd
import numpy as np
import pickle as p

In [10]:
data_all = pd.read_csv('data_all.csv')

# Test with Subset of Data
np.random.seed(511)
item_subset = list(np.random.choice(data_all['Item_ID'].unique(), size=70, replace=False))
data_subset = data_all.loc[data_all['Item_ID'].isin(item_subset)].copy()

# # Test with All Data
# data_subset = data_all.copy()

In [11]:
item_subset

[5662,
 1230,
 5820,
 1002,
 3743,
 2736,
 1203,
 3772,
 16018,
 3768,
 3766,
 7010,
 3822,
 202,
 2701,
 4975,
 5074,
 7009,
 200,
 4272,
 3737,
 4273,
 7078,
 3045,
 2721,
 3731,
 12,
 3823,
 204,
 1221,
 31,
 3742,
 2313,
 1003,
 4725,
 5068,
 4410,
 3718,
 7109,
 16000,
 7159,
 1231,
 7113,
 3736,
 102,
 1205,
 7079,
 3721,
 2867,
 2315,
 5073,
 197,
 7967,
 7008,
 7128,
 7158,
 2257,
 7129,
 1502,
 2731,
 7114,
 21,
 7575,
 6750,
 7434,
 1152,
 3716,
 4270,
 3818,
 5069]

### `/train`

In [12]:
import requests
import json

url = 'https://fillet.azurewebsites.net/train/'

data = data_subset.to_json()

payload = {'data': data,
           'cv_acc': True,
           'project_id': 16
           }

headers = {'content-type': 'application/json',
           'Accept-Charset': 'UTF-8'
          }

payload = json.dumps(payload)
r = requests.post(url, data=payload, headers=headers, timeout=1200)

In [13]:
r.json()['result']

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [ ]:
# CV Accuracies Returned
cv_accuracies = pd.DataFrame.from_dict((r.json()['cv_acc']))
cv_accuracies

### `/optimize`

In [ ]:
url = 'http://localhost:5000/optimize/'

payload = {'project_id': 1,
           'constraints': [],
           'population': 100,
           'max_epoch': 200
           }

headers = {'content-type': 'application/json',
           'Accept-Charset': 'UTF-8'
          }

payload = json.dumps(payload)
r = requests.post(url, data=payload, headers=headers)

In [ ]:
best_prices = r.json()['result']
best_prices

In [ ]:
import requests
import json

url = 'http://localhost:5000/predict/'
payload = {'prices': best_prices,
           'project_id': 1
           }
headers = {'content-type': 'application/json',
           'Accept-Charset': 'UTF-8'
          }
payload = json.dumps(payload)
r = requests.post(url, data=payload, headers=headers)

# What If Prediction
qty = r.json()['qty_estimates']

In [ ]:
qty

In [ ]:
# Calculate Revenue
revenue = 0
for item in best_prices.keys():
    revenue += (best_prices[item]*qty['Qty_'+item])
    
print(revenue)

### `/predict/`

In [14]:
# Sample Actual Price 
from helper_functions import *

sales_data = data_subset

sales_data = optimize_memory(sales_data)

sales_data_wide = sales_data.set_index(
    ['Wk', 'Tier', 'Store', 'Item_ID']).unstack(level=-1).reset_index().copy()
sales_data_wide.columns = [
    ''.join(str(i) for i in col).strip()
    for col in sales_data_wide.columns.values
]
sales_data_wide = sales_data_wide.sort_values(
    ['Tier', 'Store', 'Wk'], ascending=True).reset_index(drop=True)

sales_data_wide_clean = sales_data_wide.dropna(axis=0).copy()
dataset = sales_data_wide_clean
price_columns = [
    col for col in sales_data_wide_clean.columns if col.startswith('Price')
]

a = dataset.sample(1)
a_dict = a[price_columns].iloc[0].to_dict()
a_input = {}
for key in a_dict.keys():
    a_input[key.split('_')[1]]=round(a_dict[key],2)

print('Input Prices Quantities:')
a_input

Input Prices Quantities:


{'12': 9.4,
 '21': 3.8,
 '31': 2.1,
 '102': 9.55,
 '197': 10.4,
 '200': 5.5,
 '202': 2.0,
 '204': 9.8,
 '1002': 2.6,
 '1003': 3.0,
 '1152': 3.9,
 '1203': 3.2,
 '1205': 4.15,
 '1221': 3.45,
 '1230': 3.4,
 '1231': 4.15,
 '1502': 3.65,
 '2257': 11.4,
 '2313': 8.85,
 '2315': 12.5,
 '2701': 5.7,
 '2721': 12.65,
 '2731': 7.8,
 '2736': 9.8,
 '2867': 5.55,
 '3045': 2.4,
 '3716': 6.0,
 '3718': 7.45,
 '3721': 10.0,
 '3731': 10.7,
 '3736': 8.2,
 '3737': 8.9,
 '3742': 10.3,
 '3743': 11.0,
 '3766': 9.4,
 '3768': 10.8,
 '3772': 9.5,
 '3818': 10.3,
 '3822': 10.95,
 '3823': 11.65,
 '4270': 7.8,
 '4272': 12.1,
 '4273': 12.8,
 '4410': 4.9,
 '4725': 8.4,
 '4975': 7.65,
 '5068': 8.95,
 '5069': 9.65,
 '5073': 3.7,
 '5074': 6.7,
 '5662': 0.7,
 '5820': 2.45,
 '6750': 3.4,
 '7008': 3.8,
 '7009': 4.5,
 '7010': 4.85,
 '7078': 4.45,
 '7079': 5.0,
 '7109': 4.35,
 '7113': 3.9,
 '7114': 4.4,
 '7128': 4.2,
 '7129': 4.75,
 '7158': 5.2,
 '7159': 5.85,
 '7434': 5.35,
 '7575': 3.7,
 '7967': 4.7,
 '16000': 6.75,
 '16018'

In [18]:
# Predict Sales Qty and Compare to Actual

import requests
import json

url = 'https://fillet.azurewebsites.net/predict/'

payload = {'prices': a_input, 'project_id': 16}

headers = {'content-type': 'application/json', 'Accept-Charset': 'UTF-8'}

payload = json.dumps(payload)

r = requests.post(url, data=payload, headers=headers)
print(r.status_code)
qty = r.json()['qty_estimates']
qty_outp = {}
for key in qty.keys():
    qty_outp[key] = round(qty[key], 0)
actual = a[[
    col for col in sales_data_wide_clean.columns if col.startswith('Qty_')
]].iloc[0].to_dict()
pd.DataFrame([qty_outp, actual], index=['Estimated', 'Actual']).transpose()

JSONDecodeError: Expecting value: line 1 column 1 (char 0)